In [1]:
def simpleMatch(a, b):
    return 1 if a == b else -1

def distanceMatch(a, b):
    return 0 if a == b else -1

def linearGap(n):
    return -1 * n

def alignmentScore(s1, s2, gapPenalty, match):
    if not s1 or not s2:
        return gapPenalty(len(s1)) + gapPenalty(len(s2))
    else:
        return max(gapPenalty(1) + alignmentScore(s1, s2[1:], gapPenalty, match), 
                   gapPenalty(1) + alignmentScore(s1[1:], s2, gapPenalty, match),
                   match(s1[0], s2[0]) + alignmentScore(s1[1:], s2[1:], gapPenalty, match)) 

In [ ]:
alignmentScore("ATG", "GACT", linearGap, simpleMatch)

In [ ]:
alignmentScore("ATG", "GACT", lambda n: -1 * n, lambda a, b: 1 if a == b else -1)

In [ ]:
alignmentScore("ATG", "GACT", lambda n: 0, lambda a, b: 1 if a == b else 0)

In [2]:
def alignmentScoreOutput(s1, s2, gapPenalty, match):
    if not s1 or not s2:
        return (s1 + '-' * len(s2), s2 + '-' * len(s1), 
                gapPenalty(len(s1)) + gapPenalty(len(s2)))
    else:
        s1d, s2d, dd = alignmentScoreOutput(s1, s2[1:], gapPenalty, match)
        dd += gapPenalty(1)
        s1i, s2i, di = alignmentScoreOutput(s1[1:], s2, gapPenalty, match)
        di += gapPenalty(1) 
        s1r, s2r, dr = alignmentScoreOutput(s1[1:], s2[1:], gapPenalty, match)
        dr += match(s1[0], s2[0])
        if dd == max(dd, di, dr):
            return '-' + s1d, s2[0] + s2d, dd
        elif di == max(dd, di, dr):
            return s1[0] + s1i, '-' + s2i, di
        elif dr == max(dd, di, dr):
            return s1[0] + s1r, (s2[0] if s2[0] == s1[0] else s2[0].lower()) + s2r, dr
        else:
            assert False
            
def showAlignment(s1, s2, gapPenalty, match):
    r = alignmentScoreOutput(s1, s2, gapPenalty, match)
    print (r[0] + "\n" + r[1] + "\n" + str(r[2]))
    return r

In [3]:
showAlignment("ATG", "GACT", linearGap, simpleMatch)

-A-TG
GACT-
-1


('-A-TG', 'GACT-', -1)

In [ ]:
showAlignment("TGCAGTCA", "ATGCATGCG", gapPenalty=lambda n: 0, match=simpleMatch)

In [ ]:
showAlignment("TGCAGTCA", "ATGCATGCG", gapPenalty=linearGap, match=lambda a, b: 0)

In [4]:
import numpy as np

def alignmentScoreDP(s1, s2, gapPenalty, match):
    m = np.zeros((len(s1) + 1, len(s2) + 1))
    m[0, 0] = 0
    for i in range(1, len(s1) + 1):
        m[i, 0] = gapPenalty(i)
    for j in range(1, len(s2) + 1):
        m[0, j] = gapPenalty(j)
    for i in range(1, len(s1) + 1):
        for j in range(1, len(s2) + 1):
            m[i, j] = max(gapPenalty(1) + m[i, j - 1],  
                          gapPenalty(1) + m[i - 1, j],    
                          match(s1[i - 1], s2[j - 1]) + m[i - 1, j - 1]) 
    return m
    
def readAlignment(s1, s2, m, gapPenalty, match):
    i = len(s1)
    j = len(s2)
    s1a = ""
    s2a = "" 
    score = 0
    while i > 0 or j > 0:
        if i > 0 and j > 0 and m[i, j] == m[i - 1, j - 1] + match(s1[i - 1], s2[j - 1]):
            i = i - 1
            j = j - 1
            score += match(s1[i], s2[j])
            s1a = s1[i] + s1a
            if s1[i] == s2[j]:
                s2a = s2[j] + s2a
            else:
                s2a = s2[j].lower() + s2a
        elif i > 0 and m[i, j] == m[i - 1, j] + gapPenalty(1):
            i = i - 1
            score += gapPenalty(1)
            s1a = s1[i] + s1a
            s2a = '-' + s2a
        elif j > 0 and m[i, j] == m[i, j - 1] + gapPenalty(1):
            j = j - 1
            score += gapPenalty(1)
            s1a = '-' + s1a
            s2a = s2[j] + s2a
        else:
            assert False
    return (s1a, s2a, score)

def showAlignment(s1, s2, gapPenalty, match):
    m = alignmentScoreDP(s1, s2, gapPenalty, match)
    r = readAlignment(s1, s2, m, gapPenalty, match)
    print (r[0] + "\n" + r[1] + "\n" + str(r[2]))
    return (m, r)

In [5]:
alignmentScoreDP("ATG", "GACT", linearGap, simpleMatch)

array([[ 0., -1., -2., -3., -4.],
       [-1., -1.,  0., -1., -2.],
       [-2., -2., -1., -1.,  0.],
       [-3., -1., -2., -2., -1.]])

In [6]:
r = showAlignment("GATT", "GCAT", linearGap, simpleMatch)

G-ATT
GCA-T
1


In [7]:
r = showAlignment("GCATGCG", "GATTACA", linearGap, simpleMatch)

GCA-TGCG
G-ATTaCa
0


In [8]:
r = showAlignment("GCATGCG", "GATTACA", linearGap, simpleMatch)

GCA-TGCG
G-ATTaCa
0


In [9]:
r = showAlignment("GCATGCG", "GATTACA", lambda n: -2 * n, simpleMatch)

GCATGCG
GatTaCa
-1


In [10]:
r = showAlignment("GCATGCG", "GATTACA", lambda n: 0, match = simpleMatch)

GCA-T-GC-G
G-ATTA-CA-
4


In [ ]:
import numpy as np
from itertools import chain

def alignmentScoreDPG(s1, s2, gapPenalty, match):
    m = np.zeros((len(s1) + 1, len(s2) + 1))
    m[0, 0] = 0
    for i in range(1, len(s1) + 1):
        m[i, 0] = gapPenalty(i)
    for j in range(1, len(s2) + 1):
        m[0, j] = gapPenalty(j)
    for i in range(1, len(s1) + 1):
        for j in range(1, len(s2) + 1):         
            m[i, j] = max(chain((gapPenalty(g) + m[i, j - g] for g in range(1, j)),
                                (gapPenalty(g) + m[i - g, j] for g in range(1, i)),   
                                [(match(s1[i - 1], s2[j - 1]) + m[i - 1, j - 1])]))
    return m
    
def readAlignmentG(s1, s2, m, gapPenalty, match):
    i = len(s1)
    j = len(s2)
    s1a = ""
    s2a = ""
    score = 0
    while i > 0 or j > 0:
        if i > 0 and j > 0 and m[i, j] == m[i - 1, j - 1] + match(s1[i - 1], s2[j - 1]):
            i = i - 1
            j = j - 1
            s1a = s1[i] + s1a
            s2a = (s2[j] if s1[i] == s2[j] else s2[j].lower()) + s2a
            score += match(s1[i], s2[j])
        else:
            foundit = False
            for g in range(1, i + 1):
                if m[i, j] == m[i - g, j] + gapPenalty(g):
                    s1a = s1[i - g:i] + s1a
                    s2a = ('-' * g) + s2a
                    i = i - g
                    score += gapPenalty(g)
                    foundit = True
                    break
            if not foundit:
                for g in range(1, j + 1):
                    if m[i, j] == m[i, j - g] + gapPenalty(g):
                        s1a = ('-' * g) + s1a
                        s2a = s2a[j - g:j] + s2a
                        j = j - g
                        score += gapPenalty(g)
                        foundit = True
                        break
            assert foundit
    return (s1a, s2a, score)

def showAlignmentG(s1, s2, gapPenalty, match):
    m = alignmentScoreDPG(s1, s2, gapPenalty, match)
    r = readAlignmentG(s1, s2, m, gapPenalty, match)
    print (r[0] + "\n" + r[1] + "\n" + str(r[2]))
    return (m, r)

In [ ]:
def affineGap(n, gp = -1, gn = -0.2):
    return gp + (n - 1) * gn

In [ ]:
r = showAlignment("GCATGCG", "GATTACA", linearGap, simpleMatch)

In [ ]:
r = showAlignmentG("GCATGCG", "GATTACA", linearGap, simpleMatch)

In [ ]:
r = showAlignment("AAAGAATTCA", "AAATCA", linearGap, simpleMatch)

In [ ]:
r = showAlignmentG("AAAGAATTCA", "AAATCA", linearGap, simpleMatch)

In [ ]:
r = showAlignmentG("AAAGAATTCA", "AAATCA", affineGap, simpleMatch)

In [ ]:
showAlignment("AAAGAATTCA", "AAATCA", linearGap, simpleMatch)

In [ ]:
# https://www.ncbi.nlm.nih.gov/nuccore/NM_000558.5?report=fasta (human)
hemoglobin_s2 = "ACTCTTCTGGTCCCCACAGACTCAGAGAGAACCCACCATGGTGCTGTCTCCTGCCGACAAGACCAACGTCAAGGCCGCCTGGGGTAAGGTCGGCGCGCACGCTGGCGAGTATGGTGCGGAGGCCCTGGAGAGGATGTTCCTGTCCTTCCCCACCACCAAGACCTACTTCCCGCACTTCGACCTGAGCCACGGCTCTGCCCAGGTTAAGGGCCACGGCAAGAAGGTGGCCGACGCGCTGACCAACGCCGTGGCGCACGTGGACGACATGCCCAACGCGCTGTCCGCCCTGAGCGACCTGCACGCGCACAAGCTTCGGGTGGACCCGGTCAACTTCAAGCTCCTAAGCCACTGCCTGCTGGTGACCCTGGCCGCCCACCTCCCCGCCGAGTTCACCCCTGCGGTGCACGCCTCCCTGGACAAGTTCCTGGCTTCTGTGAGCACCGTGCTGACCTCCAAATACCGTTAAGCTGGAGCCTCGGTGGCCATGCTTCTTGCCCCTTGGGCCTCCCCCCAGCCCCTCCTCCCCTTCCTGCACCCGTACCCCCGTGGTCTTTGAATAAAGTCTGAGTGGGCGGCA"

In [ ]:
# https://www.ncbi.nlm.nih.gov/nuccore/NM_001042626.1?report=fasta (troglodytes)
hemoglobin_s1 = "ACTCTTCTGGTCCCCACAGACTCAGAAAGAACCCACCATGGTGCTGTCTCCTGCCGACAAGACCAACGTCAAGGCCGCCTGGGGTAAGGTCGGCGCGCACGCTGGCGAGTATGGTGCGGAGGCCCTGGAGAGGATGTTCCTGTCCTTCCCCACCACCAAGACCTACTTCCCCCACTTCGACCTGAGCCACGGCTCTGCCCAGGTTAAGGGTCACGGCAAGAAGGTGGCCGACGCGCTGACCAACGCCGTGGCGCACGTGGACGACATGCCCAACGCGCTGTCCGCCCTGAGTGACCTGCACGCGCACAAGCTTCGGGTGGACCCGGTCAACTTCAAGCTCCTAAGCCACTGCCTGCTGGTGACCCTGGCCGCCCACCTCCCCGCCGAGTTCACCCCTGCGGTGCACGCCTCCCTGGACAAGTTCCTGGCTTCTGTGAGCACCGTGCTGACCTCCAAATACCGTTAAGCTGGAGCCTCGGTGGCCATGCTTCTTGCCCCTTGGGCCTCTCGCCAGGCCCTCCTCTCCTTCCTGCACCTGTACCCCCCCTGGTCTTTGAATAAAGTCTGAGTGGGCGGC"

In [ ]:
# https://www.ncbi.nlm.nih.gov/nuccore/XM_045784453.1?report=fasta (ursus)
hemoglobin_s3 = "AAATGCTGGCGCACTCCCCGCCCCGCACATTTCTGGTCCTCACAGACTCAGAAAGAAGCCACCATGGTGCTGTCTCCCGCCGACAAGAGCAACGTCAAGGCCACCTGGGATAAGATTGGCAGCCACGCTGGCGAGTATGGCGGCGAGGCTCTGGAGAGGACCTTCGCGTCCTTCCCCACCACCAAGACCTACTTCCCCCACTTCGACCTGAGCCCTGGCTCCGCCCAGGTCAAGGCCCACGGCAAGAAGGTGGCCGACGCCCTGACCACCGCCGCGGGCCACCTGGACGACCTGCCGGGCGCCCTGTCCGCTCTGAGCGACCTGCACGCGCACAAGCTGCGAGTGGACCCGGTCAACTTCAAGTTCCTGAGCCACTGCCTGCTGGTGACCCTGGCCAGCCACCACCCCGCGGAGTTCACCCCTGCCGTCCACGCCTCCCTGGACAAGTTCTTCAGCGCCGTGAGCACCGTGCTCACCTCCAAATACCGTTAAGCTGGAGCCGCGCGACCCTCCCGCTCCCGGCCTGGGGCCTCTTGCGCTCCGCGCACCTGAACTTCCCGATCTTTGAATAAAGTCTGAGTGGGCTGCA"

In [ ]:
_ = showAlignmentG(hemoglobin_s1, hemoglobin_s2, affineGap, distanceMatch)

In [ ]:
_ = showAlignmentG(hemoglobin_s1, hemoglobin_s3, affineGap, distanceMatch)

In [ ]:
_ = showAlignmentG(hemoglobin_s2, hemoglobin_s3, affineGap, distanceMatch)